In [ ]:


#include <stdio.h>

// CUDA kernel for vector addition
__global__ void vectorAdd(float *a, float *b, float *c, int n) {
    int idx = threadIdx.x + blockIdx.x * blockDim.x;
    if (idx < n) {
        c[idx] = a[idx] + b[idx];
    }
}

int main() {
    int n = 1024; // Vector size
    size_t size = n * sizeof(float);

    // Allocate memory on host
    float *h_a = (float *)malloc(size);
    float *h_b = (float *)malloc(size);
    float *h_c = (float *)malloc(size);

    // Initialize vectors
    for (int i = 0; i < n; i++) {
        h_a[i] = i * 1.0f;
        h_b[i] = i * 2.0f;
    }

    // Allocate memory on GPU
    float *d_a, *d_b, *d_c;
    cudaMalloc((void **)&d_a, size);
    cudaMalloc((void **)&d_b, size);
    cudaMalloc((void **)&d_c, size);

    // Copy data from host to device
    cudaMemcpy(d_a, h_a, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, h_b, size, cudaMemcpyHostToDevice);

    // Launch kernel with 256 threads per block
    int threadsPerBlock = 256;
    int blocksPerGrid = (n + threadsPerBlock - 1) / threadsPerBlock;
    vectorAdd<<<blocksPerGrid, threadsPerBlock>>>(d_a, d_b, d_c, n);

    // Copy result back to host
    cudaMemcpy(h_c, d_c, size, cudaMemcpyDeviceToHost);

    // Verify results
    for (int i = 0; i < n; i++) {
        if (h_c[i] != h_a[i] + h_b[i]) {
            printf("Error at index %d: %f != %f\n", i, h_c[i], h_a[i] + h_b[i]);
            break;
        }
    }
    printf("Vector addition successful!\n");

    // Free memory
    free(h_a);
    free(h_b);
    free(h_c);
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);

    return 0;
}



In [ ]:

nvcc vector_add.cu -o vector_add

./vector_add



In [ ]:

#include <stdio.h>
#include <cuda_runtime.h>

// CUDA kernel for matrix multiplication
__global__ void matMul(float *A, float *B, float *C, int N) {
    int row = blockIdx.y * blockDim.y + threadIdx.y; // Row index
    int col = blockIdx.x * blockDim.x + threadIdx.x; // Column index

    if (row < N && col < N) {
        float value = 0.0f;
        for (int k = 0; k < N; k++) {
            value += A[row * N + k] * B[k * N + col];
        }
        C[row * N + col] = value;
    }
}

int main() {
    int N = 512; // Matrix size (N x N)
    size_t size = N * N * sizeof(float);

    // Allocate memory on host
    float *h_A = (float *)malloc(size);
    float *h_B = (float *)malloc(size);
    float *h_C = (float *)malloc(size);

    // Initialize matrices
    for (int i = 0; i < N * N; i++) {
        h_A[i] = 1.0f; // You can use random values here
        h_B[i] = 1.0f;
    }

    // Allocate memory on GPU
    float *d_A, *d_B, *d_C;
    cudaMalloc((void **)&d_A, size);
    cudaMalloc((void **)&d_B, size);
    cudaMalloc((void **)&d_C, size);

    // Copy data from host to device
    cudaMemcpy(d_A, h_A, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, h_B, size, cudaMemcpyHostToDevice);

    // Kernel launch parameters
    dim3 threadsPerBlock(16, 16); // 16x16 = 256 threads per block
    dim3 blocksPerGrid((N + threadsPerBlock.x - 1) / threadsPerBlock.x,
                       (N + threadsPerBlock.y - 1) / threadsPerBlock.y);

    // Launch kernel
    matMul<<<blocksPerGrid, threadsPerBlock>>>(d_A, d_B, d_C, N);

    // Copy result back to host
    cudaMemcpy(h_C, d_C, size, cudaMemcpyDeviceToHost);

    // Verify results
    printf("Matrix C (first 10 elements):\n");
    for (int i = 0; i < 10; i++) {
        printf("%f ", h_C[i]);
    }
    printf("\n");

    // Free memory
    free(h_A);
    free(h_B);
    free(h_C);
    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);

    return 0;
}



In [ ]:

Save this as matrix_mul.cu

nvcc matrix_mul.cu -o matrix_mul


./matrix_mul



In [ ]:

#include <stdio.h>
#include <cuda_runtime.h>
#include <math.h>

// Activation function: ReLU
__device__ float relu(float x) {
    return x > 0 ? x : 0;
}

// CUDA kernel for matrix multiplication
__global__ void matMul(float *A, float *B, float *C, int M, int N, int K) {
    int row = blockIdx.y * blockDim.y + threadIdx.y; // Row index of C
    int col = blockIdx.x * blockDim.x + threadIdx.x; // Column index of C

    if (row < M && col < K) {
        float value = 0.0f;
        for (int i = 0; i < N; i++) {
            value += A[row * N + i] * B[i * K + col];
        }
        C[row * K + col] = value;
    }
}

// CUDA kernel for applying ReLU
__global__ void applyReLU(float *input, int size) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < size) {
        input[idx] = relu(input[idx]);
    }
}

int main() {
    // Define network dimensions
    int input_size = 512;  // Number of input features
    int hidden_size = 256; // Number of hidden neurons
    int output_size = 10;  // Number of output neurons (e.g., classes)
    int batch_size = 128;  // Number of samples in a batch

    // Allocate memory for host
    size_t input_bytes = batch_size * input_size * sizeof(float);
    size_t hidden_bytes = batch_size * hidden_size * sizeof(float);
    size_t output_bytes = batch_size * output_size * sizeof(float);

    float *h_input = (float *)malloc(input_bytes);
    float *h_weights1 = (float *)malloc(input_size * hidden_size * sizeof(float));
    float *h_weights2 = (float *)malloc(hidden_size * output_size * sizeof(float));
    float *h_hidden = (float *)malloc(hidden_bytes);
    float *h_output = (float *)malloc(output_bytes);

    // Initialize inputs and weights with random values
    for (int i = 0; i < batch_size * input_size; i++) h_input[i] = (float)rand() / RAND_MAX;
    for (int i = 0; i < input_size * hidden_size; i++) h_weights1[i] = (float)rand() / RAND_MAX;
    for (int i = 0; i < hidden_size * output_size; i++) h_weights2[i] = (float)rand() / RAND_MAX;

    // Allocate memory on device
    float *d_input, *d_weights1, *d_weights2, *d_hidden, *d_output;
    cudaMalloc((void **)&d_input, input_bytes);
    cudaMalloc((void **)&d_weights1, input_size * hidden_size * sizeof(float));
    cudaMalloc((void **)&d_weights2, hidden_size * output_size * sizeof(float));
    cudaMalloc((void **)&d_hidden, hidden_bytes);
    cudaMalloc((void **)&d_output, output_bytes);

    // Copy data from host to device
    cudaMemcpy(d_input, h_input, input_bytes, cudaMemcpyHostToDevice);
    cudaMemcpy(d_weights1, h_weights1, input_size * hidden_size * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_weights2, h_weights2, hidden_size * output_size * sizeof(float), cudaMemcpyHostToDevice);

    // Kernel launch parameters
    dim3 threadsPerBlock(16, 16);
    dim3 blocksPerGridHidden((hidden_size + threadsPerBlock.x - 1) / threadsPerBlock.x,
                             (batch_size + threadsPerBlock.y - 1) / threadsPerBlock.y);
    dim3 blocksPerGridOutput((output_size + threadsPerBlock.x - 1) / threadsPerBlock.x,
                             (batch_size + threadsPerBlock.y - 1) / threadsPerBlock.y);

    // Forward pass: Input -> Hidden Layer
    matMul<<<blocksPerGridHidden, threadsPerBlock>>>(d_input, d_weights1, d_hidden, batch_size, input_size, hidden_size);
    cudaDeviceSynchronize();

    // Apply ReLU activation on hidden layer
    int hidden_size_total = batch_size * hidden_size;
    int threadsPerBlockReLU = 256;
    int blocksPerGridReLU = (hidden_size_total + threadsPerBlockReLU - 1) / threadsPerBlockReLU;
    applyReLU<<<blocksPerGridReLU, threadsPerBlockReLU>>>(d_hidden, hidden_size_total);
    cudaDeviceSynchronize();

    // Forward pass: Hidden Layer -> Output Layer
    matMul<<<blocksPerGridOutput, threadsPerBlock>>>(d_hidden, d_weights2, d_output, batch_size, hidden_size, output_size);
    cudaDeviceSynchronize();

    // Copy results back to host
    cudaMemcpy(h_output, d_output, output_bytes, cudaMemcpyDeviceToHost);

    // Print first 10 outputs
    printf("Output (first 10 elements):\n");
    for (int i = 0; i < 10; i++) {
        printf("%f ", h_output[i]);
    }
    printf("\n");

    // Free memory
    free(h_input);
    free(h_weights1);
    free(h_weights2);
    free(h_hidden);
    free(h_output);
    cudaFree(d_input);
    cudaFree(d_weights1);
    cudaFree(d_weights2);
    cudaFree(d_hidden);
    cudaFree(d_output);

    return 0;
}



In [ ]:

Save as feedforward_nn.cu

nvcc feedforward_nn.cu -o feedforward_nn

./feedforward_nn



In [ ]:

#include <stdio.h>
#include <cuda_runtime.h>
#include <math.h>

// Activation function: ReLU and its derivative
__device__ float relu(float x) {
    return x > 0 ? x : 0;
}

__device__ float relu_derivative(float x) {
    return x > 0 ? 1 : 0;
}

// CUDA kernel for matrix multiplication
__global__ void matMul(float *A, float *B, float *C, int M, int N, int K) {
    int row = blockIdx.y * blockDim.y + threadIdx.y; // Row index of C
    int col = blockIdx.x * blockDim.x + threadIdx.x; // Column index of C

    if (row < M && col < K) {
        float value = 0.0f;
        for (int i = 0; i < N; i++) {
            value += A[row * N + i] * B[i * K + col];
        }
        C[row * K + col] = value;
    }
}

// CUDA kernel for applying ReLU
__global__ void applyReLU(float *input, int size) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < size) {
        input[idx] = relu(input[idx]);
    }
}

// CUDA kernel for computing the derivative of ReLU
__global__ void applyReLU_derivative(float *input, float *grad, int size) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < size) {
        grad[idx] *= relu_derivative(input[idx]);
    }
}

// CUDA kernel for element-wise subtraction (used for loss computation)
__global__ void subtract(float *a, float *b, float *c, int size) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < size) {
        c[idx] = a[idx] - b[idx];
    }
}

// CUDA kernel for scalar multiplication (gradient descent update)
__global__ void scalarMultiplyAdd(float *a, float *b, float scalar, int size) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < size) {
        a[idx] -= scalar * b[idx];
    }
}

int main() {
    // Define network dimensions
    int input_size = 512;  // Number of input features
    int hidden_size = 256; // Number of hidden neurons
    int output_size = 10;  // Number of output neurons (e.g., classes)
    int batch_size = 128;  // Number of samples in a batch

    float learning_rate = 0.01;

    // Allocate memory for host
    size_t input_bytes = batch_size * input_size * sizeof(float);
    size_t hidden_bytes = batch_size * hidden_size * sizeof(float);
    size_t output_bytes = batch_size * output_size * sizeof(float);

    float *h_input = (float *)malloc(input_bytes);
    float *h_weights1 = (float *)malloc(input_size * hidden_size * sizeof(float));
    float *h_weights2 = (float *)malloc(hidden_size * output_size * sizeof(float));
    float *h_hidden = (float *)malloc(hidden_bytes);
    float *h_output = (float *)malloc(output_bytes);
    float *h_target = (float *)malloc(output_bytes); // Target outputs
    float *h_output_grad = (float *)malloc(output_bytes);
    float *h_hidden_grad = (float *)malloc(hidden_bytes);

    // Initialize inputs, weights, and targets
    for (int i = 0; i < batch_size * input_size; i++) h_input[i] = (float)rand() / RAND_MAX;
    for (int i = 0; i < input_size * hidden_size; i++) h_weights1[i] = (float)rand() / RAND_MAX;
    for (int i = 0; i < hidden_size * output_size; i++) h_weights2[i] = (float)rand() / RAND_MAX;
    for (int i = 0; i < batch_size * output_size; i++) h_target[i] = (float)rand() / RAND_MAX;

    // Allocate memory on device
    float *d_input, *d_weights1, *d_weights2, *d_hidden, *d_output, *d_target, *d_output_grad, *d_hidden_grad;
    cudaMalloc((void **)&d_input, input_bytes);
    cudaMalloc((void **)&d_weights1, input_size * hidden_size * sizeof(float));
    cudaMalloc((void **)&d_weights2, hidden_size * output_size * sizeof(float));
    cudaMalloc((void **)&d_hidden, hidden_bytes);
    cudaMalloc((void **)&d_output, output_bytes);
    cudaMalloc((void **)&d_target, output_bytes);
    cudaMalloc((void **)&d_output_grad, output_bytes);
    cudaMalloc((void **)&d_hidden_grad, hidden_bytes);

    // Copy data from host to device
    cudaMemcpy(d_input, h_input, input_bytes, cudaMemcpyHostToDevice);
    cudaMemcpy(d_weights1, h_weights1, input_size * hidden_size * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_weights2, h_weights2, hidden_size * output_size * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_target, h_target, output_bytes, cudaMemcpyHostToDevice);

    // Kernel launch parameters
    dim3 threadsPerBlock(16, 16);
    dim3 blocksPerGridHidden((hidden_size + threadsPerBlock.x - 1) / threadsPerBlock.x,
                             (batch_size + threadsPerBlock.y - 1) / threadsPerBlock.y);
    dim3 blocksPerGridOutput((output_size + threadsPerBlock.x - 1) / threadsPerBlock.x,
                             (batch_size + threadsPerBlock.y - 1) / threadsPerBlock.y);

    // Training loop
    for (int epoch = 0; epoch < 100; epoch++) {
        // Forward pass: Input -> Hidden Layer
        matMul<<<blocksPerGridHidden, threadsPerBlock>>>(d_input, d_weights1, d_hidden, batch_size, input_size, hidden_size);
        cudaDeviceSynchronize();
        applyReLU<<<(batch_size * hidden_size + 255) / 256, 256>>>(d_hidden, batch_size * hidden_size);
        cudaDeviceSynchronize();

        // Forward pass: Hidden Layer -> Output Layer
        matMul<<<blocksPerGridOutput, threadsPerBlock>>>(d_hidden, d_weights2, d_output, batch_size, hidden_size, output_size);
        cudaDeviceSynchronize();

        // Compute loss gradient: output_grad = output - target
        subtract<<<(batch_size * output_size + 255) / 256, 256>>>(d_output, d_target, d_output_grad, batch_size * output_size);
        cudaDeviceSynchronize();

        // Backpropagation: Hidden Grad = Output Grad * W2^T
        matMul<<<blocksPerGridHidden, threadsPerBlock>>>(d_output_grad, d_weights2, d_hidden_grad, batch_size, output_size, hidden_size);
        cudaDeviceSynchronize();

        // Apply ReLU derivative to hidden gradients
        applyReLU_derivative<<<(batch_size * hidden_size + 255) / 256, 256>>>(d_hidden, d_hidden_grad, batch_size * hidden_size);
        cudaDeviceSynchronize();

        // Gradient Descent: Update weights
        scalarMultiplyAdd<<<(input_size * hidden_size + 255) / 256, 256>>>(d_weights1, d_hidden_grad, learning_rate, input_size * hidden_size);
        scalarMultiplyAdd<<<(hidden_size * output_size + 255) / 256, 256>>>(d_weights2, d_output_grad, learning_rate, hidden_size * output_size);
    }

    printf("Training complete!\n");

    // Free memory
    free(h_input);
    free(h_weights1);
    free(h_weights2);
    free(h_hidden);
    free(h_output);
    free(h_target);
    cudaFree(d_input);
    cudaFree(d_weights1);
    cudaFree(d_weights2);
    cudaFree(d_hidden);
    cudaFree(d_output);
    cudaFree(d_target);
    cudaFree(d_output_grad);
    cudaFree(d_hidden_grad);

    return 0;
}



In [ ]:

neural_net_training.cu.

nvcc neural_net_training.cu -o neural_net_training

./neural_net_training



In [ ]:

#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <cuda_runtime.h>

#define NUM_CLASSES 3
#define INPUT_FEATURES 4
#define HIDDEN_LAYER1 8
#define HIDDEN_LAYER2 8
#define BATCH_SIZE 150
#define EPOCHS 1000
#define LEARNING_RATE 0.01

// CUDA kernel for matrix multiplication
__global__ void matMul(float *A, float *B, float *C, int M, int N, int K) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    if (row < M && col < K) {
        float value = 0.0f;
        for (int i = 0; i < N; i++) {
            value += A[row * N + i] * B[i * K + col];
        }
        C[row * K + col] = value;
    }
}

// CUDA kernel for applying ReLU activation
__global__ void applyReLU(float *input, int size) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < size) {
        input[idx] = input[idx] > 0 ? input[idx] : 0;
    }
}

// CUDA kernel for applying ReLU derivative
__global__ void applyReLU_derivative(float *input, float *grad, int size) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < size) {
        grad[idx] *= input[idx] > 0 ? 1.0f : 0.0f;
    }
}

// CUDA kernel for softmax activation
__global__ void applySoftmax(float *input, float *output, int rows, int cols) {
    int idx = threadIdx.x + blockIdx.x * blockDim.x;

    if (idx < rows) {
        float max_val = -INFINITY, sum = 0.0f;

        // Find max value for numerical stability
        for (int j = 0; j < cols; j++) {
            max_val = fmaxf(max_val, input[idx * cols + j]);
        }

        // Compute softmax
        for (int j = 0; j < cols; j++) {
            output[idx * cols + j] = expf(input[idx * cols + j] - max_val);
            sum += output[idx * cols + j];
        }
        for (int j = 0; j < cols; j++) {
            output[idx * cols + j] /= sum;
        }
    }
}

// CUDA kernel for computing cross-entropy loss gradient
__global__ void computeLossGradient(float *output, float *target, float *grad, int size) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < size) {
        grad[idx] = output[idx] - target[idx];
    }
}

// CUDA kernel for updating weights
__global__ void updateWeights(float *weights, float *grad, float scalar, int size) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < size) {
        weights[idx] -= scalar * grad[idx];
    }
}

// Helper function to normalize features and one-hot encode targets
void loadIrisDataset(float *inputs, float *targets) {
    // Iris dataset (features and labels)
    float raw_data[150][4] = {
        {5.1, 3.5, 1.4, 0.2}, {4.9, 3.0, 1.4, 0.2}, {4.7, 3.2, 1.3, 0.2}, // Truncated
        {7.0, 3.2, 4.7, 1.4}, {6.4, 3.2, 4.5, 1.5}, {6.9, 3.1, 4.9, 1.5}, // Versicolor
        {6.3, 3.3, 6.0, 2.5}, {5.8, 2.7, 5.1, 1.9}, {7.1, 3.0, 5.9, 2.1}  // Virginica
        // Add the rest of the 150 entries
    };
    int labels[150] = {
        0, 0, 0, 1, 1, 1, 2, 2, 2  // Truncated, 0 = Setosa, 1 = Versicolor, 2 = Virginica
        // Add the rest of the 150 labels
    };

    // Normalize and one-hot encode
    for (int i = 0; i < 150; i++) {
        for (int j = 0; j < 4; j++) {
            inputs[i * 4 + j] = raw_data[i][j] / 10.0f;  // Normalize to [0, 1]
        }
        for (int k = 0; k < 3; k++) {
            targets[i * 3 + k] = (labels[i] == k) ? 1.0f : 0.0f;  // One-hot encoding
        }
    }
}

int main() {
    // Allocate host memory
    size_t input_size = BATCH_SIZE * INPUT_FEATURES * sizeof(float);
    size_t hidden1_size = BATCH_SIZE * HIDDEN_LAYER1 * sizeof(float);
    size_t hidden2_size = BATCH_SIZE * HIDDEN_LAYER2 * sizeof(float);
    size_t output_size = BATCH_SIZE * NUM_CLASSES * sizeof(float);

    float *h_inputs = (float *)malloc(input_size);
    float *h_targets = (float *)malloc(output_size);
    float *h_hidden1 = (float *)malloc(hidden1_size);
    float *h_hidden2 = (float *)malloc(hidden2_size);
    float *h_outputs = (float *)malloc(output_size);

    float *h_weights1 = (float *)malloc(INPUT_FEATURES * HIDDEN_LAYER1 * sizeof(float));
    float *h_weights2 = (float *)malloc(HIDDEN_LAYER1 * HIDDEN_LAYER2 * sizeof(float));
    float *h_weights3 = (float *)malloc(HIDDEN_LAYER2 * NUM_CLASSES * sizeof(float));

    // Initialize dataset and weights
    loadIrisDataset(h_inputs, h_targets);
    for (int i = 0; i < INPUT_FEATURES * HIDDEN_LAYER1; i++) h_weights1[i] = (float)rand() / RAND_MAX;
    for (int i = 0; i < HIDDEN_LAYER1 * HIDDEN_LAYER2; i++) h_weights2[i] = (float)rand() / RAND_MAX;
    for (int i = 0; i < HIDDEN_LAYER2 * NUM_CLASSES; i++) h_weights3[i] = (float)rand() / RAND_MAX;

    // Allocate GPU memory
    float *d_inputs, *d_targets, *d_hidden1, *d_hidden2, *d_outputs;
    float *d_weights1, *d_weights2, *d_weights3, *d_hidden_grad2, *d_hidden_grad1, *d_output_grad;

    cudaMalloc((void **)&d_inputs, input_size);
    cudaMalloc((void **)&d_targets, output_size);
    cudaMalloc((void **)&d_hidden1, hidden1_size);
    cudaMalloc((void **)&d_hidden2, hidden2_size);
    cudaMalloc((void **)&d_outputs, output_size);

    cudaMalloc((void **)&d_weights1, INPUT_FEATURES * HIDDEN_LAYER1 * sizeof(float));
    cudaMalloc((void **)&d_weights2, HIDDEN_LAYER1 * HIDDEN_LAYER2 * sizeof(float));
    cudaMalloc((void **)&d_weights3, HIDDEN_LAYER2 * NUM_CLASSES * sizeof(float));

    cudaMalloc((void **)&d_hidden_grad2, hidden2_size);
    cudaMalloc((void **)&d_hidden_grad1, hidden1_size);
    cudaMalloc((void **)&d_output_grad, output_size);

    // Copy data to GPU
    cudaMemcpy(d_inputs, h_inputs, input_size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_targets, h_targets, output_size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_weights1, h_weights1, INPUT_FEATURES * HIDDEN_LAYER1 * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_weights2, h_weights2, HIDDEN_LAYER1 * HIDDEN_LAYER2 * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_weights3, h_weights3, HIDDEN_LAYER2 * NUM_CLASSES * sizeof(float), cudaMemcpyHostToDevice);

    // Training loop
    dim3 threadsPerBlock(16, 16);
    dim3 blocksPerGrid1((HIDDEN_LAYER1 + 15) / 16, (BATCH_SIZE + 15) / 16);
    dim3 blocksPerGrid2((HIDDEN_LAYER2 + 15) / 16, (BATCH_SIZE + 15) / 16);
    dim3 blocksPerGrid3((NUM_CLASSES + 15) / 16, (BATCH_SIZE + 15) / 16);

    for (int epoch = 0; epoch < EPOCHS; epoch++) {
        // Forward pass: Input -> Hidden1
        matMul<<<blocksPerGrid1, threadsPerBlock>>>(d_inputs, d_weights1, d_hidden1, BATCH_SIZE, INPUT_FEATURES, HIDDEN_LAYER1);
        applyReLU<<<(BATCH_SIZE * HIDDEN_LAYER1 + 255) / 256, 256>>>(d_hidden1, BATCH_SIZE * HIDDEN_LAYER1);

        // Hidden1 -> Hidden2
        matMul<<<blocksPerGrid2, threadsPerBlock>>>(d_hidden1, d_weights2, d_hidden2, BATCH_SIZE, HIDDEN_LAYER1, HIDDEN_LAYER2);
        applyReLU<<<(BATCH_SIZE * HIDDEN_LAYER2 + 255) / 256, 256>>>(d_hidden2, BATCH_SIZE * HIDDEN_LAYER2);

        // Hidden2 -> Output
        matMul<<<blocksPerGrid3, threadsPerBlock>>>(d_hidden2, d_weights3, d_outputs, BATCH_SIZE, HIDDEN_LAYER2, NUM_CLASSES);
        applySoftmax<<<BATCH_SIZE, NUM_CLASSES>>>(d_outputs, d_outputs, BATCH_SIZE, NUM_CLASSES);

        // Backpropagation: Compute gradients
        computeLossGradient<<<(BATCH_SIZE * NUM_CLASSES + 255) / 256, 256>>>(d_outputs, d_targets, d_output_grad, BATCH_SIZE * NUM_CLASSES);
        matMul<<<blocksPerGrid2, threadsPerBlock>>>(d_output_grad, d_weights3, d_hidden_grad2, BATCH_SIZE, NUM_CLASSES, HIDDEN_LAYER2);
        applyReLU_derivative<<<(BATCH_SIZE * HIDDEN_LAYER2 + 255) / 256, 256>>>(d_hidden2, d_hidden_grad2, BATCH_SIZE * HIDDEN_LAYER2);

        matMul<<<blocksPerGrid1, threadsPerBlock>>>(d_hidden_grad2, d_weights2, d_hidden_grad1, BATCH_SIZE, HIDDEN_LAYER2, HIDDEN_LAYER1);
        applyReLU_derivative<<<(BATCH_SIZE * HIDDEN_LAYER1 + 255) / 256, 256>>>(d_hidden1, d_hidden_grad1, BATCH_SIZE * HIDDEN_LAYER1);

        // Update weights
        updateWeights<<<(INPUT_FEATURES * HIDDEN_LAYER1 + 255) / 256, 256>>>(d_weights1, d_hidden_grad1, LEARNING_RATE, INPUT_FEATURES * HIDDEN_LAYER1);
        updateWeights<<<(HIDDEN_LAYER1 * HIDDEN_LAYER2 + 255) / 256, 256>>>(d_weights2, d_hidden_grad2, LEARNING_RATE, HIDDEN_LAYER1 * HIDDEN_LAYER2);
        updateWeights<<<(HIDDEN_LAYER2 * NUM_CLASSES + 255) / 256, 256>>>(d_weights3, d_output_grad, LEARNING_RATE, HIDDEN_LAYER2 * NUM_CLASSES);
    }

    printf("Training completed.\n");

    // Free GPU memory
    cudaFree(d_inputs);
    cudaFree(d_targets);
    cudaFree(d_hidden1);
    cudaFree(d_hidden2);
    cudaFree(d_outputs);
    cudaFree(d_weights1);
    cudaFree(d_weights2);
    cudaFree(d_weights3);
    cudaFree(d_hidden_grad1);
    cudaFree(d_hidden_grad2);
    cudaFree(d_output_grad);

    // Free host memory
    free(h_inputs);
    free(h_targets);
    free(h_hidden1);
    free(h_hidden2);
    free(h_outputs);
    free(h_weights1);
    free(h_weights2);
    free(h_weights3);

    return 0;
}

